In [1]:
import time
import pandas as pd
from graph_utils import Graph
from agent import Agent
from predator import Predator
from prey import Prey
import math
import random
import numpy as np
import probability as prob
import pickle
import matplotlib.pyplot as plt
import networkx as nx

def create_adj_matrix(adjacency_list) -> list:

    matrix = np.zeros((len(adjacency_list), len(adjacency_list)))

    for node in adjacency_list.keys():
        for i in adjacency_list[node]:
            matrix[node][i] = 1

    return matrix

def visualise_graph(networkx_graph,color_map,filename):  
    pos = nx.nx_agraph.graphviz_layout(networkx_graph, prog="neato") # k=5/math.sqrt(G.order())
    nx.draw(networkx_graph, node_color = color_map, pos=pos, with_labels=True)

    # Just save a PNG for the plot
    plt.gcf().set_size_inches(11.2775330396, 7.04845814978) # The MacbookPro 13.3 inches size
    plt.savefig(filename, dpi=227) # The MacbookPro 13.3 inches dpi
    plt.show()
    plt.show()



In [ ]:
# total_vertices = 50
# number_of_graphs = 5

# for i in range(number_of_graphs):
#     G = Graph()
#     G.create_graph_tcol(total_vertices)
    

#     file_name = "./Graphs/graph" + str(i)  + '/graph.pkl'
#     with open(file_name, 'rb') as f:
#         loaded_dict = pickle.dump(G.graph,f)
    
#     # Store picture of graph
#     file_name2 = "./Graphs/graph" + str(i)  + '/output.png'
#     adj_matrix = create_adj_matrix(G.graph)
#     networkx_G = nx.from_numpy_matrix(adj_matrix)
#     color_map = ['lightblue' for i in range(50)]

#     visualise_graph(networkx_G,color_map,file_name2)

In [2]:
graph_number = 0
file_name = "./Graphs/graph" + str(graph_number)  + '/graph.pkl'
G = Graph()

with open(file_name, 'rb') as f:
    loaded_dict = pickle.load(f)
    G.graph = loaded_dict

print(G.graph)

{0: [1, 49, 2], 1: [0, 2, 4], 2: [1, 3, 0], 3: [2, 4, 6], 4: [3, 5, 1], 5: [4, 6, 7], 6: [5, 7, 3], 7: [6, 8, 5], 8: [7, 9, 13], 9: [8, 10, 12], 10: [9, 11, 15], 11: [10, 12, 14], 12: [11, 13, 9], 13: [12, 14, 8], 14: [13, 15, 11], 15: [14, 16, 10], 16: [15, 17, 19], 17: [16, 18, 20], 18: [17, 19, 22], 19: [18, 20, 16], 20: [19, 21, 17], 21: [20, 22, 25], 22: [21, 23, 18], 23: [22, 24, 28], 24: [23, 25, 29], 25: [24, 26, 21], 26: [25, 27, 31], 27: [26, 28, 30], 28: [27, 29, 23], 29: [28, 30, 24], 30: [29, 31, 27], 31: [30, 32, 26], 32: [31, 33, 34], 33: [32, 34, 35], 34: [33, 35, 32], 35: [34, 36, 33], 36: [35, 37, 39], 37: [36, 38, 40], 38: [37, 39, 43], 39: [38, 40, 36], 40: [39, 41, 37], 41: [40, 42], 42: [41, 43, 46], 43: [42, 44, 38], 44: [43, 45, 48], 45: [44, 46, 49], 46: [45, 47, 42], 47: [46, 48], 48: [47, 49, 44], 49: [48, 0, 45]}


In [3]:
import math

def update_prey_trans_matrix(graph: Graph) -> None:
        """
        Updates prey transition matrix P(i, j) = Probability of prey at node i at (t + 1) given prey was at node j at (t) i.e. [j -> i]
        """
        graph_nodes = graph.node_list()

        prey_transition_matrix = np.zeros((len(graph_nodes), len(graph_nodes)))

        for jnode in graph_nodes:

            neighbors = list(graph.neighbors(jnode))
            prob_each_neighbor = 1 / (len(neighbors) + 1)
    
            # When prey stays at the same node
            prey_transition_matrix[jnode, jnode] = prob_each_neighbor

            # When prey moves to a different node
            for inode in neighbors:
                prey_transition_matrix[inode, jnode] = prob_each_neighbor
        
        return prey_transition_matrix


def update_dist_pred_trans_matrix(agent_pos, graph: Graph) -> None:
        """
        Updates predator transition matrix P(i, j) = Probability of predator at node i at (t + 1) given predator was at node j at (t) i.e. [j -> i]
        (for a distracted predator)
        """
        graph_nodes = graph.node_list()

        pred_transition_matrix = np.zeros((len(graph_nodes), len(graph_nodes)))

        for jnode in graph_nodes:
            min_path_length = graph.shortest_path_length(jnode, agent_pos)
            likely_nodes_plan = []
            likely_nodes_random = graph.neighbors(jnode)

            for neighbor in graph.neighbors(jnode):
                
                path_length = graph.shortest_path_length(neighbor, agent_pos)
                
                
                if path_length <= min_path_length:
                    if path_length < min_path_length:
                        min_path_length = path_length
                        likely_nodes_plan = [neighbor]
                    else:
                        likely_nodes_plan.append(neighbor)

            for inode in likely_nodes_random:
                # When predator moves to a 'good' node (whether it is because of focus or distraction) 
                if inode in likely_nodes_plan:
                    pred_transition_matrix[inode, jnode] = 0.6 / len(likely_nodes_plan) + 0.4 / len(likely_nodes_random)
                else: # when predator is distracted and goes to a 'bad' node
                    pred_transition_matrix[inode, jnode] = 0.4 / len(likely_nodes_random)
        
        return  pred_transition_matrix
"""
This function estimates the initial utility of the state 
Good guess for estimating the utility of a state is the distance between the agent and the prey
"""

def create_all_states(total_vertices)-> list:

    all_states = []  
    # state_utility = {}
    # state_policy = {}
    for agent in range(total_vertices):
        for prey_p in range(total_vertices):
                for predator_p in range(total_vertices):
                    current_state = (agent,prey_p,predator_p)
                    all_states.append(current_state)
    return all_states

# initial guess

def create_initial_state_utility(graph: Graph, all_states)-> dict:

    state_utility = {}
    for s in all_states:
        # Shortest Distance from Agent to prey
        agent_to_prey = graph.shortest_path_length(s[0] , s[1])

        # Shortest Distance from Agent to Predator
        agent_to_predator = graph.shortest_path_length(s[0] , s[2])

        if s[0] == s[2]:
            state_utility[s] = 500000000
            
        elif s[0] == s[1]:
            state_utility[s] = 0
        
        elif agent_to_prey == 1 and s[0] != s[2]:
            state_utility[s] = 0       

        elif agent_to_predator == 1:
            state_utility[s] = 5000000

        else:
            state_utility[s] = agent_to_prey

    return state_utility


def create_initial_state_policy(graph: Graph,all_states)-> dict:

    
    state_policy = {}
    for s in all_states:
        # Shortest Distance from Agent to prey
        agent_to_prey = graph.shortest_path_length(s[0] , s[1])

        # Shortest Distance from Agent to Predator
        agent_to_predator = graph.shortest_path_length(s[0] , s[2])

        if s[0] == s[1] or s[0] == s[2]:
            state_policy[s] = s[0]
            

        elif agent_to_prey == 1 and s[0] != s[2]:
            state_policy[s] = s[1]
            

        elif agent_to_predator == 1:

            agent_neighbours = graph.neighbors(s[0])
            dis_to_pred = {}
            for i in agent_neighbours:
                dis_to_pred[i] = graph.shortest_path_length(i , s[2])

            max_val = max(dis_to_pred.values())
            best_neighbors = [neighbor for neighbor, dist in dis_to_pred.items() if dist == max_val]      
            state_policy[s] = random.choice(best_neighbors) 
            
        else:
            agent_neighbours = graph.neighbors(s[0])
            state_policy[s] = random.choice(agent_neighbours) 

    return state_policy


def create_actions(graph: Graph, all_states) -> dict:

    return {s: graph.neighbors(s[0]) for s in all_states }


def create_rewards(all_states)-> dict:
    rewards = {}
    for s in all_states:
        if s[0] == s[1] or s[0] == s[2]:
            rewards[s] = 0
        else:
            rewards[s] = 1

    return rewards



def create_transisition_model(graph: Graph, all_states, actions, prey_transition_matrix)-> dict:

    """Transition model From a state and an action, return a list
        of (probability, next_state) pairs."""

    transition_model = {}

    count = 0
    for s in all_states:
        transition_model[s] = {}

        if count%2500 == 0:
            print(count)
        
        count += 1
        for a in actions[s]:
            pred_transition_matrix = update_dist_pred_trans_matrix(a, graph)
            transition_model[s][a] = []
            # List of Neighbors of Prey (i.e help us determine neighbouring states of the system)
            prey_neighbours = graph.neighbors(s[1]).copy()
            prey_neighbours.append(s[1])
            # List of Neighbors of Predator (i.e help us determine neighbouring states of the system)
            predator_neighbours = graph.neighbors(s[2])
            
            for p in prey_neighbours:
                for pred in predator_neighbours:
                    next_state = (a,p,pred)
                    transition_probability = prey_transition_matrix[p][s[1]]*pred_transition_matrix[pred][s[2]]
                    transition_model[s][a].append((transition_probability,next_state))

            # prey_neighbours.remove(s[1])

    return transition_model



def value_iteration(graph: Graph, all_states, actions, state_utility, state_policy, transition_model, immediate_reward, beta, error_threshold) -> dict:
    """  """
    iteration = 0
    
    while True:
        print("Iteration no: "+str(iteration))

        old_state_utility = state_utility.copy()

        error = 0
        # count = 0

        for s in all_states:
            
            # if count%2500==0:
            #     print(count)
        
            # count+=1

            # Shortest Distance from Agent to prey
            agent_to_prey = graph.shortest_path_length(s[0] , s[1])

            # Shortest Distance from Agent to Predator
            agent_to_predator = graph.shortest_path_length(s[0] , s[2])

            # What states s are easy to determine U∗ for?
              
            if s[0] == s[1] or s[0] == s[2]: # 1. The terminal states
                continue
            
            
            elif agent_to_prey == 1 and s[0] != s[2] : # 2. When we are lucky enough to start next to the prey 
                continue

            elif agent_to_predator == 1:
                continue

            else: # UPDATE utility
            
            
                # print("Utility of current state {} BEFORE update: {}".format(s,old_state_utility[s]))
                
                expected_utility = {}

                for a in actions[s]:
                    expected_future_utility = 0
                    for (p, s1) in transition_model[s][a]:
                        expected_future_utility += p * state_utility[s1]
                    expected_utility[a] = expected_future_utility

                # Set Minimum reward value out of all the actions as the utility of the current state
                min_val = min(expected_utility.values())

                # UPDATE utility
                state_utility[s] = immediate_reward[s] + beta * min_val

                

                
                # print("Utility of current state {} AFTER update: {}".format(s,state_utility[s]))

                # Update policy 

                lowest_utility_neighbors = [action for action, utility in expected_utility.items() if utility == min_val]      

                # Breaking ties at random
                next_pos = random.choice(lowest_utility_neighbors)

                state_policy[s] = next_pos

                
                error = max(error, abs(state_utility[s] - old_state_utility[s]))
                

        iteration +=1
        print("Error:  "+str(error))
        
        if error <= error_threshold :
            return state_utility





In [4]:
total_vertices = 50
all_states = create_all_states(total_vertices)
state_utility = create_initial_state_utility(G,all_states)
state_policy = create_initial_state_policy(G,all_states)
actions = create_actions(G,all_states)
immediate_reward = create_rewards(all_states)

prey_transition_matrix = update_prey_trans_matrix(G)

GENERATE TRANSITION MODEL FOR THE GRAPH

In [18]:
# transition_model = create_transisition_model(G,all_states,actions,prey_transition_matrix)

0
2500
5000
7500
10000
12500
15000
17500
20000
22500
25000
27500
30000
32500
35000
37500
40000
42500
45000
47500
50000
52500
55000
57500
60000
62500
65000
67500
70000
72500
75000
77500
80000
82500
85000
87500
90000
92500
95000
97500
100000
102500
105000
107500
110000
112500
115000
117500
120000
122500


STORE the TRANSITION MODEL 

In [19]:
# file_name = "./Graphs/graph" + str(graph_number)  + '/Transition_Model.pkl'
# with open(file_name, 'wb') as f:
#     loaded_dict = pickle.dump(transition_model,f)

READ THE TRANSITION MODEL FOR THE PARTICULAR GRAPH

In [5]:
file_name = "./Graphs/graph" + str(graph_number)  + '/Transition_Model.pkl'

with open(file_name, 'rb') as f:
    transition_model = pickle.load(f)


GENERATE OPTIMAL STATE UTILITY

In [6]:
beta = 1
error_threshold = 0.0001

optimal_state_utility = value_iteration(G, all_states, actions, state_utility, state_policy, transition_model, immediate_reward, beta, error_threshold)


Iteration no: 0
Error:  366666666.1333332
Iteration no: 1
Error:  2688886.6066089254
Iteration no: 2
Error:  780320.9560309784
Iteration no: 3
Error:  104937.84613369487
Iteration no: 4
Error:  90842.35431295598
Iteration no: 5
Error:  33099.86212387045
Iteration no: 6
Error:  6521.225305882741
Iteration no: 7
Error:  1391.1964013105753
Iteration no: 8
Error:  296.7836131723161
Iteration no: 9
Error:  63.321091559562774
Iteration no: 10
Error:  13.655186942480213
Iteration no: 11
Error:  3.147094180654676
Iteration no: 12
Error:  1.1170657082361473
Iteration no: 13
Error:  0.868680189604337
Iteration no: 14
Error:  0.7136960727778856
Iteration no: 15
Error:  0.5180259426687854
Iteration no: 16
Error:  0.4190514636805531
Iteration no: 17
Error:  0.3651519808924846
Iteration no: 18
Error:  0.3231767330809987
Iteration no: 19
Error:  0.28632850937319887
Iteration no: 20
Error:  0.2480682661076301
Iteration no: 21
Error:  0.21842508588050435
Iteration no: 22
Error:  0.1916566665210233
Iter

STORE the Optimal State Utility for the particular graph

PKL file 

In [17]:
# file_name = "./Graphs/graph" + str(graph_number)  + '/Optimal_su.pkl'
# with open(file_name, 'wb') as f:
#     loaded_dict = pickle.dump(optimal_state_utility,f)

# file_name = "./Graphs/graph" + str(graph_number)  + '/Optimal_policy.pkl'
# with open(file_name, 'wb') as f:
#     loaded_dict = pickle.dump(state_policy,f)

CSV file

In [18]:
# df = pd.DataFrame()

# a_p = []
# p_p = []
# pred_p = []
# o_p = []
# d = []
# o = []
# for i in list(optimal_state_utility.keys()):
#     a_p.append(i[0])
#     p_p.append(i[1])
#     pred_p.append(i[2])
#     d.append(G.shortest_path_length(i[0],i[1])/2)
#     o.append(G.shortest_path_length(i[0],i[2])/2)

    
    
# df["Agent_position"] = a_p
# df["Prey_position"] = p_p
# df["Predator_position"]= pred_p
# df["Optimal_Policy"] = state_policy.values()
# df["Optimal_utility"] = state_utility.values()
# df['Dist. b/w Agent and Prey'] = d
# df['Dist. b/w Agent and Predator'] = o

# file_name = "./Graphs/graph" + str(graph_number)  + '/Optimal_su.csv'
# df.to_csv(file_name, encoding='utf-8')

READ the Optimal state utility for the particular graph

In [2]:
file_name = "./Graphs/graph" + str(graph_number)  + '/Optimal_su.pkl'

with open(file_name, 'rb') as f:
    optimal_state_utility = pickle.load(f)

READ the Optimal state policy for the particular graph

In [15]:
file_name = "./Graphs/graph" + str(graph_number)  + '/Optimal_policy.pkl'
with open(file_name, 'rb') as f:
    state_policy = pickle.load(f)

In [7]:
import math
import random
import numpy as np
import probability as prob
from graph_utils import Graph

class Agent:
    """
    Object that has stuff on the agent
    """
    def __init__(self, name, graph: Graph,pos) -> None:
        """
        Initialization
        """
        self.name = name

        # Spawn Agent at a random node (other than the prey and predator spawn nodes)
        self.node = pos
        # random.choice(graph.node_list())

        total_nodes = len(graph.node_list())

        # Initialize prey probability matrix to (1/49)s
        self.prey_beliefs = []

        for _ in range(total_nodes):
            self.prey_beliefs.append(1 / (total_nodes - 1))

        self.prey_beliefs[self.node] = 0.0

        # Initialize predator probability matrix to 0s
        self.pred_beliefs = []

        for _ in range(total_nodes):
            self.pred_beliefs.append(0.0)

        # For partial prey
        self.prey_transition_matrix = np.zeros((total_nodes, total_nodes), dtype=float)

        # For partial predator
        self.pred_transition_matrix = np.zeros((total_nodes, total_nodes), dtype=float)

        print("Initial Agent Position: " + str(self.node))

    def prey_move_sim(self, graph: Graph, prey_pos, agent_future_pos) -> int:
        """
        The function simulates the predator's movement pattern (used in Agents 2, 4, 6 and 8)
        """
        possible_positions = [prey_pos]
        possible_positions.extend(graph.neighbors(prey_pos))

        path_lengths = {}

        for prey in possible_positions:
            path_lengths[prey] = graph.shortest_path_length(prey, agent_future_pos)

        # Just set the current position to the best neighbor (and break ties randomly if there is more than one)
        return round(sum(list(path_lengths.values())) / len(possible_positions))

    def predator_move_sim(self, graph: Graph, pred_pos, agent_future_pos, distracted=False) -> int:
        """
        The function simulates the predator's movement pattern (used in Agents 2, 4, 6 and 8)
        """
        min_path_len = math.inf

        for neighbor in graph.neighbors(pred_pos):
            
            path_length = graph.shortest_path_length(neighbor, agent_future_pos)
            
            if path_length < min_path_len:
                min_path_len = path_length # this is done because we have a clear cut best neighbor

        # Just set the current position to the best neighbor (and break ties randomly if there is more than one)
        return path_length

    def update_prey_trans_matrix(self, graph: Graph) -> None:
        """
        Updates prey transition matrix P(i, j) = Probability of prey at node i at (t + 1) given prey was at node j at (t) i.e. [j -> i]
        """
        graph_nodes = graph.node_list()

        for jnode in graph_nodes:

            neighbors = list(graph.neighbors(jnode))
            prob_each_neighbor = 1 / (len(neighbors) + 1)
    
            # When prey stays at the same node
            self.prey_transition_matrix[jnode, jnode] = prob_each_neighbor

            # When prey moves to a different node
            for inode in neighbors:
                self.prey_transition_matrix[inode, jnode] = prob_each_neighbor

    def update_pred_trans_matrix(self, graph: Graph) -> None:
        """
        Updates predator transition matrix P(i, j) = Probability of predator at node i at (t + 1) given predator was at node j at (t) i.e. [j -> i]
        (for a non-distracted predator)
        """
        graph_nodes = graph.node_list()

        self.pred_transition_matrix = np.zeros((len(graph_nodes), len(graph_nodes)))

        for jnode in graph_nodes:
            min_path_length = math.inf
            likely_nodes_plan = []

            for neighbor in graph.neighbors(jnode):
                
                path_length = graph.shortest_path_length(jnode, self.node)
                
                if path_length <= min_path_length:
                    if path_length < min_path_length:
                        min_path_length = path_length
                    likely_nodes_plan.append(neighbor)

            for inode in likely_nodes_plan:
                self.pred_transition_matrix[inode, jnode] = 1 / len(likely_nodes_plan)


    def update_dist_pred_trans_matrix(self, graph: Graph) -> None:
        """
        Updates predator transition matrix P(i, j) = Probability of predator at node i at (t + 1) given predator was at node j at (t) i.e. [j -> i]
        (for a distracted predator)
        """
        graph_nodes = graph.node_list()

        self.pred_transition_matrix = np.zeros((len(graph_nodes), len(graph_nodes)))

        for jnode in graph_nodes:
            min_path_length = math.inf
            likely_nodes_plan = []
            likely_nodes_random = graph.neighbors(jnode)

            for neighbor in graph.neighbors(jnode):
                
                path_length = graph.shortest_path_length(jnode, self.node)
                
                if path_length <= min_path_length:
                    if path_length < min_path_length:
                        min_path_length = path_length
                        likely_nodes_plan = [neighbor]
                    else:
                        likely_nodes_plan.append(neighbor)

            for inode in likely_nodes_random:
                # When predator moves to a 'good' node (whether it is because of focus or distraction) 
                if inode in likely_nodes_plan:
                    self.pred_transition_matrix[inode, jnode] = 0.6 / len(likely_nodes_plan) + 0.4 / len(likely_nodes_random)
                else: # when predator is distracted and goes to a 'bad' node
                    self.pred_transition_matrix[inode, jnode] = 0.4 / len(likely_nodes_random)

    def move_1(self, graph: Graph, prey_pos, pred_pos) -> None:
        """
        This function moves the agent 1 according to the strategy mentioned in the write up
        """
        
        # Shortest Distance from Agent to prey
        agent_to_prey = graph.shortest_path_length(self.node , prey_pos)
        
        # Shortest Distance from Agent to Predator
        agent_to_predator = graph.shortest_path_length(self.node , pred_pos)

        # List of Neighbors of Agent
        agent_neighbours = graph.neighbors(self.node)

        # Distance of shortest path from each neighbour to prey and predator
        dist_neighbors = {}
        
        # Setting priority for every neighbor according to the order followed by Agent1 
        priority = {}

        for neighbor in agent_neighbours:

            # Distance of shortest path from neighbor to prey
            neighbor_to_prey = graph.shortest_path_length( neighbor, prey_pos)

            # Distance of shortest path from neighbor to predator
            neighbor_to_predator = graph.shortest_path_length( neighbor, pred_pos)

            dist_neighbors[neighbor] = (neighbor_to_prey,neighbor_to_predator)

            if neighbor_to_prey < agent_to_prey and neighbor_to_predator > agent_to_predator:
                priority[neighbor] = 1
                
            elif neighbor_to_prey < agent_to_prey and neighbor_to_predator == agent_to_predator :
                priority[neighbor] = 2
                
            elif neighbor_to_prey == agent_to_prey and neighbor_to_predator > agent_to_predator :
                priority[neighbor] = 3
                
            elif neighbor_to_prey == agent_to_prey and neighbor_to_predator == agent_to_predator :
                priority[neighbor] = 4
                
            elif neighbor_to_predator > agent_to_predator :
                priority[neighbor] = 5
                
            elif neighbor_to_predator == agent_to_predator :
                priority[neighbor] = 6
                
            else :
                priority[neighbor] = 7
        
        # Highest priority value out of all the neighbors
        min_val = min(priority.values())

        # Neighbors with the Highest priority
        if min_val < 7:
            # List of neighbors with highest priority
            Highest_priority_neighbors = [key for key, value in priority.items() if value == min_val]      

            # Breaking ties at random
            next_pos = random.choice(Highest_priority_neighbors)
            
            # Update node for the agent
            self.node = next_pos

        else:
            self.node = self.node
    
    def move_2(self, graph: Graph, prey_pos, pred_pos, distracted=False) -> None:
        """
        Movement logic for Agent 2
        """
        # Shortest Distance from Agent to prey
        agent_to_prey = graph.shortest_path_length(self.node , prey_pos)
        
        # Shortest Distance from Agent to Predator
        agent_to_predator = graph.shortest_path_length(self.node , pred_pos)

        # List of Neighbors of Agent
        agent_neighbours = graph.neighbors(self.node)

        # Distance of shortest path from each neighbour to prey and predator
        dist_neighbors = {}

        # Setting priority for every neighbor according to the order followed by Agent1 
        priority = {}

        for neighbor in agent_neighbours:
            # Distance of shortest path from neighbor to prey
            neighbor_to_prey = graph.shortest_path_length(neighbor, prey_pos)

            # Distance of shortest path from neighbor to predator
            neighbor_to_predator = graph.shortest_path_length( neighbor, pred_pos)

            # Distance of shortest path from neighbor to predator future
            neighbor_to_predator_future = self.predator_move_sim(graph, pred_pos, neighbor, distracted) # We just need distance

            neighbor_to_prey_future = self.prey_move_sim(graph, prey_pos, neighbor) # We get an average distance

            dist_neighbors[neighbor] = (neighbor_to_prey, neighbor_to_predator)

            if neighbor_to_prey_future < agent_to_prey and neighbor_to_predator_future > agent_to_predator and neighbor_to_predator >= agent_to_predator :
                priority[neighbor] = 1

            elif neighbor_to_prey_future < agent_to_prey and neighbor_to_predator_future == agent_to_predator and neighbor_to_predator >= agent_to_predator :
                priority[neighbor] = 2
                
            elif neighbor_to_prey_future == agent_to_prey and neighbor_to_predator_future > agent_to_predator and neighbor_to_predator >= agent_to_predator :
                priority[neighbor] = 3
            
            elif neighbor_to_prey_future == agent_to_prey and neighbor_to_predator_future == agent_to_predator and neighbor_to_predator >= agent_to_predator :
                priority[neighbor] = 4

            elif neighbor_to_prey < agent_to_prey and neighbor_to_predator_future > agent_to_predator and neighbor_to_predator >= agent_to_predator :
                priority[neighbor] = 5

            elif neighbor_to_prey < agent_to_prey and neighbor_to_predator_future == agent_to_predator and neighbor_to_predator >= agent_to_predator :
                priority[neighbor] = 6
                
            elif neighbor_to_prey == agent_to_prey and neighbor_to_predator_future > agent_to_predator and neighbor_to_predator >= agent_to_predator :
                priority[neighbor] = 7
            
            elif neighbor_to_prey == agent_to_prey and neighbor_to_predator_future == agent_to_predator and neighbor_to_predator >= agent_to_predator :
                priority[neighbor] = 8

            elif neighbor_to_predator_future > agent_to_predator and neighbor_to_predator >= agent_to_predator :
                priority[neighbor] = 9

            elif neighbor_to_predator_future == agent_to_predator and neighbor_to_predator >= agent_to_predator :
                priority[neighbor] = 10

            elif neighbor_to_prey_future < agent_to_prey and neighbor_to_predator > agent_to_predator:
                priority[neighbor] = 11
                
            elif neighbor_to_prey_future < agent_to_prey and neighbor_to_predator == agent_to_predator :
                priority[neighbor] = 12

            elif neighbor_to_prey_future == agent_to_prey and neighbor_to_predator > agent_to_predator :
                priority[neighbor] = 13
                
            elif neighbor_to_prey_future == agent_to_prey and neighbor_to_predator == agent_to_predator :
                priority[neighbor] = 14

            elif neighbor_to_prey < agent_to_prey and neighbor_to_predator > agent_to_predator :
                priority[neighbor] = 15
                
            elif neighbor_to_prey < agent_to_prey and neighbor_to_predator == agent_to_predator :
                priority[neighbor] = 16

            elif neighbor_to_prey == agent_to_prey and neighbor_to_predator > agent_to_predator :
                priority[neighbor] = 17
                
            elif neighbor_to_prey == agent_to_prey and neighbor_to_predator == agent_to_predator :
                priority[neighbor] = 18

            elif neighbor_to_predator > agent_to_predator :
                priority[neighbor] = 19
                
            elif neighbor_to_predator == agent_to_predator :
                priority[neighbor] = 20

            else :
                priority[neighbor] = 21
            
        # Highest priority value out of all the neighbors
        min_val = 21
        min_val = min(priority.values())

        # Neighbors with the Highest priority
        if min_val < 21:
            # List of neighbors with highest priority
            Highest_priority_neighbors = [key for key, value in priority.items() if value == min_val]

            # Breaking ties at random
            next_pos = random.choice(Highest_priority_neighbors)

            # Update node for the agent
            self.node = next_pos
        
        else:
            self.node = self.node

    def move_3(self, graph: Graph, prey_pos: int, pred_pos: int, time_steps: int) -> bool:
        """
        Movement logic for Agents 3 and 4
        """
        error = 10 ** -5

        if time_steps == 1:
            # Initial transition matrix (not to be altered)
            self.update_prey_trans_matrix(graph)

        elif time_steps > 1:
            # Update beliefs post prey move
            """
            P ( prey at some_node now ) = SUM [ P ( prey at some_node now AND prey was at old_node then ) ]
                ... Marginalization

            P ( prey at some_node now ) = SUM [ P ( prey at old_node then ) * P ( prey at some_node now | prey at old_node then ) ]
                ... Conditional Factoring

            P ( prey at some_node now ) = SUM [ P ( prey at old_node then ) * P ( some_node | old_node ) ]
                ... Simplifying the last probability which is basically the transition probability
            
            New beilief = DOT PRODUCT [ old_belief , row in the transition matrix ]
                ... In terms of what we have
            """

            updated_beliefs_ndarray = np.matmul(self.prey_transition_matrix, np.array(self.prey_beliefs))
            self.prey_beliefs = updated_beliefs_ndarray.tolist()

            if not prob.check_sum_beliefs(self.prey_beliefs):
                raise ValueError("Sum of beliefs error (after prey move update)")

        # Find the best survey pos
        best_survey_node = prob.node_to_survey(self.prey_beliefs, "prey")

        # Perform survey on the best node
        self.prey_beliefs = prob.survey(self.prey_beliefs, best_survey_node, prey_pos)

        if not prob.check_sum_beliefs(self.prey_beliefs):
            raise ValueError("Sum of beliefs error (after node survey)")

        # Find the node for which we have highest belief
        max_prob_nodes = [node for node, belief in enumerate(self.prey_beliefs) if belief == max(self.prey_beliefs)]

        # If more than one, break ties randomly if 
        prob_prey_pos = random.choice(max_prob_nodes)

        print("Highest probable current position of prey is [{}] with probability [{}], so we'll use that info".format(prob_prey_pos, self.prey_beliefs[prob_prey_pos]))

        if self.name == "agent3":
            # Do the actual movement to highest belief node based on agent 1 logic for agent 3
            self.move_1(graph, prob_prey_pos, pred_pos)
        
        elif self.name == "agent4":
            # Do the actual movement to highest belief node based on agent 2 logic for agent 4
            self.move_2(graph, prob_prey_pos, pred_pos)

        # Update beliefs post agent move
        prob.survey(self.prey_beliefs, self.node, prey_pos)

        if not prob.check_sum_beliefs(self.prey_beliefs):
            raise ValueError("Sum of beliefs error (after agent move)")
        
        if 1.0 - error <= max(self.prey_beliefs) <= 1.0 + error:
            return True
        else:
            return False


    def move_utility(self, graph: Graph, prey_pos, pred_pos,state_policy) -> None:
        """
        This function moves the agent according to the U* strategy mentioned in the write up
        """

        state = (self.node,prey_pos,pred_pos)        
        self.node = state_policy[state] 


Non Terminal States

In [32]:
terminal_states = []
non_terminal_states = []
for s in all_states:
    if s[0] == s[1] or s[0] == s[2]:
        terminal_states.append(s)
    else:
        non_terminal_states.append(s)
print(len(terminal_states))
print(len(non_terminal_states))
print(len(non_terminal_states) + len(terminal_states))


4950
120050
125000


In [11]:
total_vertices = 50
agentNum = 11
max_steps = 5000
num_runs = 1000
num_trials = 30
print("Agent Number: "+str(agentNum))

Agent Number: 11


In [12]:
# Choose N random states from non terminal states

# selected_idx = random.sample(range(0, len(non_terminal_states)), num_runs)

# selected_states = [ non_terminal_states[i] for i in selected_idx]


STORE THE SELECTED STATES

In [13]:
# file_name = "./Graphs/graph" + str(graph_number) + "/agent_csv" + "/starting_states.pkl"
# with open(file_name, 'wb') as f:
#     loaded_dict = pickle.dump(selected_states,f)

STORE CSV FILE

In [15]:
# df = pd.DataFrame()

# a_p = []
# p_p = []
# pred_p = []
# o_p = []

# for i in selected_states :
#     a_p.append(i[0])
#     p_p.append(i[1])
#     pred_p.append(i[2])

    
    
# df["Agent_position"] = a_p
# df["Prey_position"] = p_p
# df["Predator_position"]= pred_p


# file_name = "./Graphs/graph" + str(graph_number) + "/agent_csv" + "/starting_states.csv"
# df.to_csv(file_name, encoding='utf-8')

Read the selected states

In [6]:
file_name = "./Graphs/graph" + str(graph_number) + "/agent_csv" + "/starting_states.pkl"
with open(file_name, 'rb') as f:
    selected_states = pickle.load(f)

In [18]:
import time
import pandas as pd
from graph_utils import Graph
# from agent import Agent
from predator import Predator
from prey import Prey
import random



# total_vertices = int(input("Enter the total vertices number: "))
# agentNum = int(input("Enter the agent number: "))

agentChar = ""
if agentNum == 7 or agentNum == 8:
    agentChar = str(input("Enter a space if normal agent 7 or 8, else enter b: ")).strip()

# num_runs = int(input("Enter the number of runs: "))
# num_trials = int(input("Enter the number of trials per run: "))
# max_steps = int(input("Enter the max number of steps each trial should run for: "))

start_time = time.time()

total_success = 0



# Run the agent/prey/predator game (based on input params)
for run in range(num_runs):

    print("Run number: "+ str(run))
    print()
    success = {}
    fail_predator = {}
    fail_hang = {}

    time_steps = {}
    times_prey_known = {}
    times_pred_known = {}

    agent_path = {}
    prey_path = {}
    predator_path = {}

    current_state = selected_states[run]
    

    for trial in range(num_trials):
        success[trial] = fail_predator[trial] = fail_hang[trial] = 0

    for trial in range(num_trials):

        # print("Trial number: "+ str(trial))
        # print()

        agent_path[trial] = []
        prey_path[trial] = []
        predator_path[trial] = []

        # Spawn the agent
        a = Agent("agent" + str(agentNum) + agentChar, G, current_state[0])
        agent_path[trial].append(a.node)

        # Spawn Prey and Predator
        prey = Prey(G, a,current_state[1])
        prey_path[trial].append(prey.pos)

        predator = Predator(G, a,current_state[2])
        predator_path[trial].append(predator.pos)

        # print("Initial Prey Position: "+str(prey.pos))
        # print("Initial Predator Position: "+str(predator.pos))

        # print("Initial Distance between Agent and Prey: "+str(G.shortest_path_length(a.node,prey.pos)))
        # print("Initial Distance between Agent and Predator: "+str(G.shortest_path_length(a.node,predator.pos)))
        # print()
        # By default set the agent as not caught
        caught_us = False
        caught_prey = False

        # Resetting time_steps - an upper bound of max_steps is used to make sure agent isn't running forever
        time_steps[trial] = 0
        times_prey_known[trial] = 0
        times_pred_known[trial] = 0

        while not caught_us and not caught_prey and time_steps[trial] < max_steps:

            time_steps[trial] += 1

            # MOVE AGENT
            if a.name == "agent1":
                a.move_1(G, prey.pos, predator.pos)
                agent_path[trial].append(a.node)
                times_pred_known[trial] += 1
                times_prey_known[trial] += 1
                # print("Next position of Agent: " + str(a.node))


            elif a.name == "agent2":
                a.move_2(G, prey.pos, predator.pos, distracted=False)
                agent_path[trial].append(a.node)
                times_pred_known[trial] += 1
                times_prey_known[trial] += 1
                # print("Next position of Agent: " + str(a.node))

            elif a.name == "agent3":
                if a.move_3(G, prey.pos, predator.pos, time_steps[trial]):
                    agent_path[trial].append(a.node)
                    times_prey_known[trial] += 1
                times_pred_known[trial] += 1
                # print("Next position of Agent: " + str(a.node))

            elif a.name == "agent4":
                if a.move_3(G, prey.pos, predator.pos, time_steps[trial]):
                    agent_path[trial].append(a.node)
                    times_prey_known[trial] += 1
                times_pred_known[trial] += 1
                # print("Next position of Agent: " + str(a.node))
            
            if a.name == "agent11":
                a.move_utility(G, prey.pos, predator.pos, state_policy)
                agent_path[trial].append(a.node)
                times_pred_known[trial] += 1
                times_prey_known[trial] += 1
                # print("Next position of Agent: " + str(a.node))
                # print("AGENT_to_Prey BEFORE prey move: "+str(G.shortest_path_length(a.node,prey.pos)))

            

            # CHECK IF CAUGHT OR NOT
            if a.node == predator.pos :
                caught_us = True
                fail_predator[trial] = 1
                break

            elif a.node == prey.pos :
                caught_prey = True
                success[trial] = 1
                break           

            # MOVE PREY (Randomly)
            prey.move_prey(G)
            prey_path[trial].append(prey.pos)
            # print("Next position of Prey: " + str(prey.pos))
            
            # print("AGENT_to_Prey AFTER prey move: "+str(G.shortest_path_length(a.node,prey.pos)))


            # CHECK IF CAUGHT OR NOT
            if a.node == predator.pos :
                caught_us = True
                fail_predator[trial] = 1
                break

            elif a.node == prey.pos :
                caught_prey = True
                success[trial] = 1
                break

            # MOVE PREDATOR (Shortest path to agent)
            predator.move_distracted_predator(G, a)
            predator_path[trial].append(predator.pos)
            # print("Next position of Predator: " + str(predator.pos))
            # print()

            # CHECK IF CAUGHT OR NOT
            if a.node == predator.pos:
                caught_us = True
                fail_predator[trial] = 1
                break

            elif a.node == prey.pos:
                caught_prey = True
                success[trial] = 1
                break
        
        # End of movement while loop
    
        # Check what was the reason of exiting the while loop and update the counters
        if not caught_us and not caught_prey and time_steps[trial] == max_steps:
            fail_hang[trial] = 1
        if caught_us:
            print("Predator Caught the Agent at location: "+str(a.node))
        elif caught_prey:
            print("Agent Caught the Prey at location: "+str(a.node))

        print("Total timesteps: "+str(time_steps[trial]))
        print()

        
        file_name = "./Graphs/graph" + str(graph_number)  + "/agent_csv/agent" + str(agentNum) + agentChar + "/path/Agent/Run" + str(run) + "/Trial"+ str(trial)+'.pkl'
    
        with open(file_name, 'wb') as f:
            loaded_dict = pickle.dump(agent_path[trial],f)

        file_name = "./Graphs/graph" + str(graph_number)  + "/agent_csv/agent" + str(agentNum) + agentChar + "/path/Prey/Run" + str(run) + "/Trial"+ str(trial)+'.pkl'
    
        with open(file_name, 'wb') as f:
            loaded_dict = pickle.dump(prey_path[trial],f)
        
        file_name = "./Graphs/graph" + str(graph_number)  + "/agent_csv/agent" + str(agentNum) + agentChar + "/path/Predator/Run" + str(run) + "/Trial"+ str(trial)+'.pkl'

        with open(file_name, 'wb') as f:
            loaded_dict = pickle.dump(predator_path[trial],f)

    # End of all trials


    df = pd.DataFrame()
    df["success"]= success.values()
    df["failure_predator"]= fail_predator.values()
    df["failure_timeout"]= fail_hang.values()
    df["time_steps"] = time_steps.values()

    
    file_name = "./Graphs/graph" + str(graph_number)  + "/agent_csv/agent" + str(agentNum) + agentChar + "/data/Run" + str(run) + '.csv'
    df.to_csv(file_name, encoding='utf-8')

    



    total_success += sum(list(success.values()))

    # df_2 = pd.DataFrame()
    # df_2["times_prey_known"] = times_prey_known.values()
    # df_2["times_pred_known"] = times_pred_known.values()
    # df_2["time_steps"] = time_steps.values()
    # file_name_2 = "./know_csv/agent" + str(agentNum) + agentChar + "/Run" + str(run) + '.csv'
    # df_2.to_csv(file_name_2, encoding='utf-8')

# End of runs

print("The total success percentage for the run was: [{}%]".format((total_success * 100) / (num_runs * num_trials)))

print("The time taken for {} runs ({} trials per run) was: [{}] seconds".format(num_runs, num_trials, round(time.time() - start_time, 2)))

Run number: 1000

Initial Agent Position: 29
Agent Caught the Prey at location: 32
Total timesteps: 5

Initial Agent Position: 29
Agent Caught the Prey at location: 39
Total timesteps: 32

Initial Agent Position: 29
Agent Caught the Prey at location: 49
Total timesteps: 16

Initial Agent Position: 29
Agent Caught the Prey at location: 32
Total timesteps: 30

Initial Agent Position: 29
Agent Caught the Prey at location: 35
Total timesteps: 23

Initial Agent Position: 29
Agent Caught the Prey at location: 33
Total timesteps: 11

Initial Agent Position: 29
Agent Caught the Prey at location: 37
Total timesteps: 24

Initial Agent Position: 29
Agent Caught the Prey at location: 36
Total timesteps: 12

Initial Agent Position: 29
Agent Caught the Prey at location: 39
Total timesteps: 9

Initial Agent Position: 29
Agent Caught the Prey at location: 41
Total timesteps: 23

Initial Agent Position: 29
Agent Caught the Prey at location: 37
Total timesteps: 16

Initial Agent Position: 29
Agent Caugh